## 准备数据

In [46]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [47]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [48]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 第一层784-->128
        # 第二层128-->10
        self.W1 = tf.Variable(shape=[28*28,128],dtype=tf.float32,
                              initial_value = tf.random.uniform(shape=[28*28,128],minval=-0.1,maxval=0.1))
        self.b1 = tf.Variable(shape=[128],dtype=tf.float32,
                              initial_value = tf.random.uniform(shape=[128],minval=-0.1,maxval=0.1))
        self.W2 = tf.Variable(shape=[128,10],dtype=tf.float32,
                              initial_value = tf.random.uniform(shape=[128,10],minval=-0.1,maxval=0.1))
        self.b2 = tf.Variable(shape=[10],dtype=tf.float32,
                              initial_value = tf.random.uniform(shape=[10],minval=-0.1,maxval=0.1))
        self.trainable_vars = [self.W1, self.W2, self.b1, self.b2]

        
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_reshape = tf.reshape(x,shape=[-1,28*28])
        h = tf.matmul(x_reshape,self.W1)+self.b1
        h = tf.tanh(h)
        logits = tf.matmul(h,self.W2)+self.b2
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [49]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(1*g)  # lr 从0.01改成1

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [50]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3580828 ; accuracy 0.121
epoch 1 : loss 1.8524984 ; accuracy 0.4644
epoch 2 : loss 1.4827251 ; accuracy 0.6199833
epoch 3 : loss 1.2307662 ; accuracy 0.69091666
epoch 4 : loss 1.0721161 ; accuracy 0.73145
epoch 5 : loss 0.97039557 ; accuracy 0.71195
epoch 6 : loss 0.9200746 ; accuracy 0.7342833
epoch 7 : loss 0.8224307 ; accuracy 0.73765
epoch 8 : loss 0.80096644 ; accuracy 0.76585
epoch 9 : loss 0.77099955 ; accuracy 0.7402833
epoch 10 : loss 0.7128405 ; accuracy 0.78861666
epoch 11 : loss 0.74975 ; accuracy 0.7345833
epoch 12 : loss 0.8069993 ; accuracy 0.7237667
epoch 13 : loss 0.86843336 ; accuracy 0.69846666
epoch 14 : loss 0.7306111 ; accuracy 0.7489333
epoch 15 : loss 0.609281 ; accuracy 0.79503334
epoch 16 : loss 0.5821746 ; accuracy 0.80698335
epoch 17 : loss 0.52422726 ; accuracy 0.8386
epoch 18 : loss 0.51419145 ; accuracy 0.8462667
epoch 19 : loss 0.4938594 ; accuracy 0.85305
epoch 20 : loss 0.4722739 ; accuracy 0.86468333
epoch 21 : loss 0.47161028 ; accur